<a href="https://colab.research.google.com/github/SreevishnuAB/ChimpData/blob/main/Normalise_and_Plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
!pip install pandas plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [146]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [188]:
import pandas as pd
import plotly.express as px

import plotly.graph_objects as pgo

In [148]:
df_pai_wet_ssn = pd.read_csv("drive/MyDrive/Colab Notebooks/pai_by_pairs_wet_season.csv", dtype="str")
df_pai_dry_ssn = pd.read_csv("drive/MyDrive/Colab Notebooks/pai_by_pairs_dry_season.csv", dtype="str")
df_dai_wet_ssn_ff = pd.read_csv("drive/MyDrive/Colab Notebooks/dai_wet_season_ff.csv", dtype="str")
df_dai_dry_ssn_ff = pd.read_csv("drive/MyDrive/Colab Notebooks/dai_dry_season_ff.csv", dtype="str")
df_dai_wet_ssn_mm = pd.read_csv("drive/MyDrive/Colab Notebooks/dai_wet_season_mm.csv", dtype="str")
df_dai_dry_ssn_mm = pd.read_csv("drive/MyDrive/Colab Notebooks/dai_dry_season_mm.csv", dtype="str")

In [149]:
df_pai_wet_ssn['PAI'] = pd.to_numeric(df_pai_wet_ssn['PAI'])
df_pai_dry_ssn['PAI'] = pd.to_numeric(df_pai_dry_ssn['PAI'])
df_dai_wet_ssn_ff['DAI'] = pd.to_numeric(df_dai_wet_ssn_ff['DAI'])
df_dai_dry_ssn_ff['DAI'] = pd.to_numeric(df_dai_dry_ssn_ff['DAI'])
df_dai_wet_ssn_mm['DAI'] = pd.to_numeric(df_dai_wet_ssn_mm['DAI'])
df_dai_dry_ssn_mm['DAI'] = pd.to_numeric(df_dai_dry_ssn_mm['DAI'])


In [150]:
print(df_pai_wet_ssn['PAI'].describe())
print(df_pai_dry_ssn['PAI'].describe())
print(df_dai_wet_ssn_ff['DAI'].describe())
print(df_dai_dry_ssn_ff['DAI'].describe())
print(df_dai_wet_ssn_mm['DAI'].describe())
print(df_dai_dry_ssn_mm['DAI'].describe())

count    141.000000
mean       1.683134
std        2.846962
min        0.159207
25%        0.958630
50%        1.145564
75%        1.358716
max       25.447812
Name: PAI, dtype: float64
count    256.000000
mean       1.486874
std        1.956950
min        0.268290
25%        0.854485
50%        1.103147
75%        1.342849
max       22.086452
Name: PAI, dtype: float64
count    13.000000
mean      0.221767
std       0.249851
min       0.016129
25%       0.050633
50%       0.118182
75%       0.227273
max       0.787879
Name: DAI, dtype: float64
count    52.000000
mean      0.148847
std       0.132344
min       0.006993
25%       0.071141
50%       0.114576
75%       0.182488
max       0.637931
Name: DAI, dtype: float64
count    80.000000
mean      0.279269
std       0.110218
min       0.013072
25%       0.231055
50%       0.276090
75%       0.328225
max       0.501916
Name: DAI, dtype: float64
count    144.000000
mean       0.241260
std        0.110555
min        0.023041
25%        0.1

In [151]:
def normalise_column(col: pd.Series) -> pd.Series:
  min = col.min()
  max = col.max()
  
  def normalise_value(val):
    normalised_val = (val - min) / (max - min)
    return pd.Series((normalised_val))

  return col.apply(normalise_value)

In [152]:
def normalise_column_zscore(col: pd.Series) -> pd.Series:
  mean = col.mean()
  std = col.std()
  
  def normalise_value(val):
    normalised_val = (val - mean) / std
    return pd.Series((normalised_val))

  return col.apply(normalise_value)

In [153]:
def merge_in_order(val1, val2):
  if val1 < val2:
    return pd.Series((f"{val1} - {val2}"))
  else:
    return pd.Series((f"{val2} - {val1}"))

In [154]:
df_pai_wet_ssn['NPAI'] = normalise_column(df_pai_wet_ssn['PAI'])
df_pai_wet_ssn['pair'] = df_pai_wet_ssn.apply(lambda x: merge_in_order(x.A, x.B), axis=1)
df_pai_wet_ssn

,A,B,PAI,NPAI,pair
0,alizeti,bingwa,9.520257,0.370169,alizeti - bingwa
1,almasi,bingwa,9.520257,0.370169,almasi - bingwa
2,baseka,sanaa,0.410552,0.009939,baseka - sanaa
3,bingwa,samaki,1.280400,0.044336,bingwa - samaki
4,bingwa,kinanda,1.543825,0.054753,bingwa - kinanda
...,...,...,...,...,...
136,wingu,kitambi,1.075633,0.036239,kitambi - wingu
137,wingu,rosa,0.855241,0.027524,rosa - wingu
138,zai,waiti,1.179467,0.040345,waiti - zai
139,zai,mbawa,0.793434,0.025080,mbawa - zai


In [155]:
# df_pai_wet_ssn['NPAI'] = normalise_column_zscore(df_pai_wet_ssn['PAI'])
# df_pai_wet_ssn['pair'] = df_pai_wet_ssn['A'] + " - " + df_pai_wet_ssn['B']
# df_pai_wet_ssn

In [156]:
df_pai_wet_ssn_sorted = df_pai_wet_ssn.sort_values("NPAI", ascending=False)
df_pai_wet_ssn_sorted['index'] = df_pai_wet_ssn_sorted.index.astype(str)
print(len(df_pai_wet_ssn_sorted))
df_pai_wet_ssn_sorted = df_pai_wet_ssn_sorted[['pair', 'NPAI']].drop_duplicates()
df_pai_wet_ssn_sorted

141


,pair,NPAI
45,joto - mbawa,1.000000
47,jua - mbawa,0.698811
119,baseka - unid1,0.393077
0,alizeti - bingwa,0.370169
1,almasi - bingwa,0.370169
...,...,...
89,kitambi - mbawa,0.009809
48,dhahabu - jua,0.008245
113,baseka - samaki,0.004666
102,kinanda - mosi,0.002655


In [157]:
fig = px.bar(df_pai_wet_ssn_sorted, x="pair", y="NPAI")
fig.update_layout(
    width=4000,
    height=4000,)
fig.show()

In [158]:
df_pai_dry_ssn['NPAI'] = normalise_column(df_pai_dry_ssn['PAI'])
df_pai_dry_ssn['pair'] = df_pai_dry_ssn.apply(lambda x: merge_in_order(x.A, x.B), axis=1)
df_pai_dry_ssn

,A,B,PAI,NPAI,pair
0,alizeti,jua,6.030787,0.264115,alizeti - jua
1,baseka,maisha,2.172857,0.087293,baseka - maisha
2,baseka,kinanda,0.822046,0.025381,baseka - kinanda
3,baseka,wingu,0.796241,0.024198,baseka - wingu
4,bingwa,wingu,1.168489,0.041259,bingwa - wingu
...,...,...,...,...,...
251,zai,sanaa,1.093369,0.037816,sanaa - zai
252,zai,bonge,0.317325,0.002247,bonge - zai
253,zai,kitambi,0.842799,0.026332,kitambi - zai
254,zai,jua,2.195206,0.088317,jua - zai


In [159]:
df_pai_dry_ssn_sorted = df_pai_dry_ssn.sort_values("NPAI", ascending=False)
df_pai_dry_ssn_sorted['index'] = df_pai_dry_ssn_sorted.index.astype(str)
print(len(df_pai_dry_ssn_sorted))
df_pai_dry_ssn_sorted = df_pai_dry_ssn_sorted[['pair', 'NPAI']].drop_duplicates()
df_pai_dry_ssn_sorted

256


,pair,NPAI
213,baseka - unid1,1.000000
102,kinanda - kinundi,0.567124
158,maisha - maua,0.544672
240,uyoga - zai,0.478066
75,jua - mbawa,0.306602
...,...,...
218,bonge - uyoga,0.002595
252,bonge - zai,0.002247
32,dhahabu - mbawa,0.001739
166,mbawa - wingu,0.000108


In [160]:
fig = px.bar(df_pai_dry_ssn_sorted, x="pair", y="NPAI")
fig.update_layout(
    width=4000,
    height=4000,)
fig.show()

In [161]:
df_dai_wet_ssn_ff['NDAI'] = normalise_column(df_dai_wet_ssn_ff['DAI'])
df_dai_wet_ssn_ff['pair'] = df_dai_wet_ssn_ff.apply(lambda x: merge_in_order(x.A, x.B), axis=1)
df_dai_wet_ssn_ff

,A,B,DAI,NDAI,pair
0,corona,baseka,0.121622,0.136693,baseka - corona
1,corona,jua,0.050633,0.044709,corona - jua
2,corona,kasolya,0.223810,0.269103,corona - kasolya
3,kasolya,kovujicho,0.227273,0.273591,kasolya - kovujicho
4,mbawa,baseka,0.016129,0.000000,baseka - mbawa
5,mbawa,joto,0.787879,1.000000,joto - mbawa
6,mbawa,jua,0.657895,0.831572,jua - mbawa
7,mbawa,kovujicho,0.026490,0.013425,kovujicho - mbawa
8,mbawa,maua,0.426230,0.531390,maua - mbawa
9,mbawa,zai,0.039474,0.030249,mbawa - zai


In [162]:
df_dai_wet_ssn_ff_sorted = df_dai_wet_ssn_ff.sort_values("NDAI", ascending=False)
df_dai_wet_ssn_ff_sorted['index'] = df_dai_wet_ssn_ff_sorted.index.astype(str)
print(len(df_dai_wet_ssn_ff_sorted))
df_dai_wet_ssn_ff_sorted = df_dai_wet_ssn_ff_sorted[['pair', 'NDAI']].drop_duplicates()
df_dai_wet_ssn_ff_sorted

13


,pair,NDAI
5,joto - mbawa,1.000000
6,jua - mbawa,0.831572
8,maua - mbawa,0.531390
3,kasolya - kovujicho,0.273591
2,corona - kasolya,0.269103
0,baseka - corona,0.136693
12,rosa - zai,0.132236
11,kovujicho - rosa,0.107960
10,kasolya - rosa,0.093013
1,corona - jua,0.044709


In [163]:
fig = px.bar(df_dai_wet_ssn_ff_sorted, x="pair", y="NDAI")
fig.update_layout(
    width=4000,
    height=4000,)
fig.show()

In [164]:
df_dai_dry_ssn_ff['NDAI'] = normalise_column(df_dai_dry_ssn_ff['DAI'])
df_dai_dry_ssn_ff['pair'] = df_dai_dry_ssn_ff.apply(lambda x: merge_in_order(x.A, x.B), axis=1)
df_dai_dry_ssn_ff

,A,B,DAI,NDAI,pair
0,alizeti,jua,0.006993,0.000000,alizeti - jua
1,baseka,joto,0.115942,0.172678,baseka - joto
2,baseka,jua,0.089189,0.130276,baseka - jua
3,baseka,kasolya,0.058091,0.080988,baseka - kasolya
4,baseka,kinanda,0.021898,0.023623,baseka - kinanda
5,baseka,maua,0.054475,0.075256,baseka - maua
6,baseka,zai,0.039735,0.051894,baseka - zai
7,corona,kasolya,0.202586,0.310004,corona - kasolya
8,corona,kovujicho,0.158924,0.240802,corona - kovujicho
9,jua,corona,0.076072,0.109486,corona - jua


In [165]:
df_dai_dry_ssn_ff_sorted = df_dai_dry_ssn_ff.sort_values("NDAI", ascending=False)
df_dai_dry_ssn_ff_sorted['index'] = df_dai_dry_ssn_ff_sorted.index.astype(str)
print(len(df_dai_dry_ssn_ff_sorted))
df_dai_dry_ssn_ff_sorted = df_dai_dry_ssn_ff_sorted[['pair', 'NDAI']].drop_duplicates()
len(df_dai_dry_ssn_ff_sorted)

52


35

In [166]:
fig = px.bar(df_dai_dry_ssn_ff_sorted, x="pair", y="NDAI")
fig.update_layout(
    width=4000,
    height=4000,)
fig.show()

In [167]:
df_dai_wet_ssn_mm['NDAI'] = normalise_column(df_dai_wet_ssn_mm['DAI'])
df_dai_wet_ssn_mm['pair'] = df_dai_wet_ssn_mm.apply(lambda x: merge_in_order(x.A, x.B), axis=1)
df_dai_wet_ssn_mm

,A,B,DAI,NDAI,pair
0,bingwa,dhahabu,0.248705,0.482021,bingwa - dhahabu
1,bingwa,mosi,0.124031,0.226983,bingwa - mosi
2,bingwa,samaki,0.221198,0.425752,bingwa - samaki
3,bonge,waiti,0.207792,0.398328,bonge - waiti
4,dhahabu,bingwa,0.248705,0.482021,bingwa - dhahabu
...,...,...,...,...,...
75,wingu,kitambi,0.278049,0.542048,kitambi - wingu
76,wingu,mosi,0.308411,0.604159,mosi - wingu
77,wingu,samaki,0.197183,0.376626,samaki - wingu
78,wingu,sanaa,0.366013,0.721992,sanaa - wingu


In [168]:
df_dai_wet_ssn_mm_sorted = df_dai_wet_ssn_mm.sort_values("NDAI", ascending=False)
df_dai_wet_ssn_mm_sorted['index'] = df_dai_wet_ssn_mm_sorted.index.astype(str)
print(len(df_dai_wet_ssn_mm_sorted))
df_dai_wet_ssn_mm_sorted = df_dai_wet_ssn_mm_sorted[['pair', 'NDAI']].drop_duplicates()
len(df_dai_wet_ssn_mm_sorted)

80


58

In [169]:
fig = px.bar(df_dai_wet_ssn_mm_sorted, x="pair", y="NDAI")
fig.update_layout(
    width=4000,
    height=4000,)
fig.show()

In [170]:
df_dai_dry_ssn_mm['NDAI'] = normalise_column(df_dai_dry_ssn_mm['DAI'])
df_dai_dry_ssn_mm['pair'] = df_dai_dry_ssn_mm.apply(lambda x: merge_in_order(x.A, x.B), axis=1)
df_dai_dry_ssn_mm

,A,B,DAI,NDAI,pair
0,bingwa,elisha,0.278800,0.505477,bingwa - elisha
1,bingwa,kitambi,0.258166,0.464698,bingwa - kitambi
2,bingwa,waiti,0.177778,0.305819,bingwa - waiti
3,bonge,bingwa,0.302254,0.551833,bingwa - bonge
4,bonge,dhahabu,0.246212,0.441072,bonge - dhahabu
...,...,...,...,...,...
139,wingu,mbaya,0.355856,0.657771,mbaya - wingu
140,wingu,mosi,0.223577,0.396337,mosi - wingu
141,wingu,samaki,0.378109,0.701752,samaki - wingu
142,wingu,sanaa,0.312611,0.572302,sanaa - wingu


In [171]:
df_dai_dry_ssn_mm_sorted = df_dai_dry_ssn_mm.sort_values("NDAI", ascending=False)
df_dai_dry_ssn_mm_sorted['index'] = df_dai_dry_ssn_mm_sorted.index.astype(str)
print(len(df_dai_dry_ssn_mm_sorted))
df_dai_dry_ssn_mm_sorted = df_dai_dry_ssn_mm_sorted[['pair', 'NDAI']].drop_duplicates()
len(df_dai_dry_ssn_mm_sorted)

144


89

In [172]:
fig = px.bar(df_dai_dry_ssn_mm_sorted, x="pair", y="NDAI")
fig.update_layout(
    width=4000,
    height=4000,)
fig.show()

In [232]:
# create 20 cluster; 0.00 - 0.050, 0.051 - 0.1, 0.1 - 0.15, .....

def cluster_pair(col: pd.Series) -> pd.Series:
  return col.apply(lambda val: pd.Series(((val // 0.05) + 1)))

df_pai_wet_ssn_sorted['cluster'] = cluster_pair(df_pai_wet_ssn_sorted['NPAI'])
df_pai_dry_ssn_sorted['cluster'] = cluster_pair(df_pai_dry_ssn_sorted['NPAI'])
df_dai_wet_ssn_ff_sorted['cluster'] = cluster_pair(df_dai_wet_ssn_ff_sorted['NDAI'])
df_dai_dry_ssn_ff_sorted['cluster'] = cluster_pair(df_dai_dry_ssn_ff_sorted['NDAI'])
df_dai_wet_ssn_mm_sorted['cluster'] = cluster_pair(df_dai_wet_ssn_mm_sorted['NDAI'])
df_dai_dry_ssn_mm_sorted['cluster'] = cluster_pair(df_dai_dry_ssn_mm_sorted['NDAI'])
df_pai_wet_ssn_sorted['cluster']

45     20.0
47     14.0
119     8.0
0       8.0
1       8.0
       ... 
89      1.0
48      1.0
113     1.0
102     1.0
84      1.0
Name: cluster, Length: 141, dtype: float64

In [233]:
fig = px.bar(df_pai_wet_ssn_sorted, x="pair", y="cluster")
fig.update_layout(
    width=4000,
    height=4000,)
fig.show()

In [234]:
fig = px.bar(df_pai_dry_ssn_sorted, x="pair", y="cluster")
fig.update_layout(
    width=4000,
    height=4000,)
fig.show()

In [235]:
fig = px.bar(df_dai_wet_ssn_ff_sorted, x="pair", y="cluster")
fig.update_layout(
    width=4000,
    height=4000,)
fig.show()

In [236]:
fig = px.bar(df_dai_dry_ssn_ff_sorted, x="pair", y="cluster")
fig.update_layout(
    width=4000,
    height=4000,)
fig.show()

In [237]:
fig = px.bar(df_dai_wet_ssn_mm_sorted, x="pair", y="cluster")
fig.update_layout(
    width=4000,
    height=4000,)
fig.show()

In [238]:
fig = px.bar(df_dai_dry_ssn_mm_sorted, x="pair", y="cluster")
fig.update_layout(
    width=4000,
    height=4000,)
fig.show()

In [239]:
pair_pai_dry = set(df_pai_dry_ssn_sorted['pair'].values)
pair_pai_wet = set(df_pai_wet_ssn_sorted['pair'].values)
pair_pai_dry.intersection(pair_pai_wet)

{'baseka - samaki',
 'baseka - unid1',
 'bingwa - corona',
 'bingwa - dhahabu',
 'bingwa - elisha',
 'bingwa - kasolya',
 'bingwa - kitambi',
 'bingwa - kovujicho',
 'bingwa - kukulon',
 'bingwa - mbawa',
 'bingwa - mosi',
 'bonge - dhahabu',
 'bonge - imba',
 'bonge - waiti',
 'bonge - wingu',
 'corona - dhahabu',
 'corona - elisha',
 'corona - imba',
 'corona - jua',
 'corona - kasolya',
 'corona - kitambi',
 'corona - kukulon',
 'corona - mbaya',
 'corona - mosi',
 'corona - samaki',
 'corona - waiti',
 'corona - wingu',
 'dhahabu - elisha',
 'dhahabu - imba',
 'dhahabu - jua',
 'dhahabu - kasolya',
 'dhahabu - kitambi',
 'dhahabu - kovujicho',
 'dhahabu - kukolon',
 'dhahabu - kukulon',
 'dhahabu - mosi',
 'dhahabu - samaki',
 'dhahabu - sanaa',
 'dhahabu - waiti',
 'dhahabu - wingu',
 'elisha - imba',
 'elisha - kasolya',
 'elisha - kitambi',
 'elisha - kovujicho',
 'elisha - kukolon',
 'elisha - kukulon',
 'elisha - mbaya',
 'elisha - mosi',
 'elisha - rosa',
 'elisha - samaki',


In [240]:
pair_pai_wet - pair_pai_dry

{'alizeti - bingwa',
 'almasi - bingwa',
 'baseka - corona',
 'baseka - dhahabu',
 'baseka - mbawa',
 'baseka - sanaa',
 'bingwa - kinanda',
 'bingwa - kukolon',
 'bingwa - rosa',
 'bingwa - samaki',
 'corona - kukolon',
 'corona - unid1',
 'dhahabu - rosa',
 'imba - unid1',
 'kinanda - mosi',
 'kitambi - mbawa',
 'kovujicho - mbawa',
 'kovujicho - rosa',
 'kukolon - mbawa',
 'kukolon - rosa',
 'kukulon - mosi',
 'mbawa - uyoga',
 'mbawa - waiti',
 'rosa - sanaa',
 'rosa - waiti'}

In [241]:
df_pai = pd.merge(df_pai_dry_ssn_sorted, df_pai_wet_ssn_sorted, how="inner", on=['pair', "cluster"])
df_pai = df_pai[(df_pai['NPAI_x'] > 0.009) & (df_pai['NPAI_y'] > 0.009)] # remove insignificant values
df_pai.loc[:, 'NPAI_x'] = df_pai.NPAI_x.round(3)
df_pai.loc[:, 'NPAI_y'] = df_pai.NPAI_y.round(3)
df_pai

,pair,NPAI_x,cluster,NPAI_y
0,mosi - rosa,0.069,2.0,0.067
1,sanaa - waiti,0.055,2.0,0.050
2,waiti - wingu,0.052,2.0,0.051
3,imba - kitambi,0.052,2.0,0.050
4,imba - mosi,0.050,1.0,0.048
...,...,...,...,...
77,elisha - zai,0.021,1.0,0.034
78,dhahabu - mosi,0.020,1.0,0.034
79,rosa - uyoga,0.015,1.0,0.047
80,maua - mosi,0.013,1.0,0.027


In [242]:
plot = pgo.Figure(data=[pgo.Bar(name="Dry Season", x=df_pai['pair'], y=df_pai['NPAI_x']), pgo.Bar(name="Wet Season", x=df_pai['pair'], y=df_pai['NPAI_y'])])
plot.update_layout(
    width=3000,
    height=3000,)
plot.update_yaxes(nticks=40)
plot.show()

In [245]:
df_dai_ff = pd.merge(df_dai_dry_ssn_ff_sorted, df_dai_wet_ssn_ff_sorted, how="inner", on=['pair', "cluster"])
df_dai_ff = df_dai_ff[(df_dai_ff['NDAI_x'] > 0.009) & (df_dai_ff['NDAI_y'] > 0.009)] # remove insignificant values
df_dai_ff.loc[:, 'NDAI_x'] = df_dai_ff.NDAI_x.round(3)
df_dai_ff.loc[:, 'NDAI_y'] = df_dai_ff.NDAI_y.round(3)
df_dai_ff

,pair,NDAI_x,cluster,NDAI_y
0,kasolya - kovujicho,0.277,6.0,0.274


In [248]:
plot = pgo.Figure(data=[pgo.Bar(name="Dry Season", x=df_dai_ff['pair'], y=df_dai_ff['NDAI_x']), pgo.Bar(name="Wet Season", x=df_dai_ff['pair'], y=df_dai_ff['NDAI_y'])])
plot.update_layout(
    width=1000,
    height=1000,)
plot.update_yaxes(nticks=40)
plot.show()

In [244]:
df_dai_mm = pd.merge(df_dai_dry_ssn_mm_sorted, df_dai_wet_ssn_mm_sorted, how="inner", on=['pair', "cluster"])
df_dai_mm = df_dai_mm[(df_dai_mm['NDAI_x'] > 0.009) & (df_dai_mm['NDAI_y'] > 0.009)] # remove insignificant values
df_dai_mm.loc[:, 'NDAI_x'] = df_dai_mm.NDAI_x.round(3)
df_dai_mm.loc[:, 'NDAI_y'] = df_dai_mm.NDAI_y.round(3)
df_dai_mm

,pair,NDAI_x,cluster,NDAI_y
0,imba - samaki,0.561,12.0,0.562
1,dhahabu - waiti,0.556,12.0,0.599
2,imba - waiti,0.531,11.0,0.529
3,elisha - waiti,0.511,11.0,0.534
4,kitambi - mbaya,0.400,9.0,0.450
5,bonge - waiti,0.370,8.0,0.398
6,dhahabu - kukulon,0.092,2.0,0.096


In [249]:
plot = pgo.Figure(data=[pgo.Bar(name="Dry Season", x=df_dai_mm['pair'], y=df_dai_mm["NDAI_x"]), pgo.Bar(name="Wet Season", x=df_dai_mm['pair'], y=df_dai_mm['NDAI_y'])])
plot.update_layout(
    width=2000,
    height=2000,)
plot.update_yaxes(nticks=40)
plot.show()